# icolonEpithelium and gut microbiome integration example

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd


In [ ]:
os.environ['GUROBI_HOME'] = "gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "gurobi.lic"

In [ ]:
from micom.util import load_model

colon = load_model("iColonEpithelium_vhm.xml")
colon.compartments

In [ ]:
colon_constraints = pd.read_excel('refined_medium_input.xlsx', 'Sink')

We will first make the medium very limited from the blood side by only allowing oxygen influx and will reset the objective to the maintenance function.


In [ ]:
colon_micom = colon.copy()

colon_micom.objective = colon_micom.reactions.biomass_maintenance
for i in range(len(colon_constraints)):
    sink_rxn = colon_constraints.loc[i]['Sink_reactions']
    ub = colon_constraints.loc[i]['Upper_boundary']
    lb = colon_constraints.loc[i]['Lower_boundary']
    print("(", lb, ',', ub, ")")
    if lb == 0:
        
        colon.reactions.get_by_id(sink_rxn).bounds = (lb, ub)
    else:
        colon.reactions.get_by_id(sink_rxn).bounds = (-0.1, ub)
#colon_micom.reactions.get_by_id('EX_ppa[e]').bounds = (0, 100)
colon_micom.reactions.get_by_id('EX_co2[lu]').bounds = (0, 100) 
colon_micom.reactions.get_by_id('EX_co2[e]').bounds = (0, 100)  
colon_micom.summary()

In [7]:
colon_micom.medium = {"EX_o2[e]": 100}

Now we read a MICOM model for a healthy individual.

In [ ]:
from micom.util import load_pickle  

com = load_pickle("ERR1883195.pickle")
com.taxa

Before we add the host we will add some coupling constraints for the taxa that ensures that flux can only take place if the taxa grow. That will, for instance, avoid butyrate production from taxa that don't grow. We use a combination of [resource constraints](https://pubmed.ncbi.nlm.nih.gov/34990848/) and [biomass coupling](https://pmc.ncbi.nlm.nih.gov/articles/PMC3555882/#sec14) to get a resource coupling in the form of:

$$
\sum_i |v_i| < \mu \cdot M
$$

for each taxon.

In [9]:
com.add_coupling_constraints(strategy="resource coupling", constraint=400)

We use a new function in the a developing version of MICOM to combine the MICOM model with the iColonEpithelium 

In [ ]:
com.add_host(
    model=colon_micom,
    id="colon",
    shared_compartment="lu",
    own_compartment="e",
    abundance=1
)

We apply the average europen diet medium to the model.

In [ ]:
import micom as mm
med = mm.qiime_formats.load_qiime_medium("western_diet_gut_agora.qza")
com.medium = med.flux

Now including the host in the cooperative tradeoff funtion of MICOM. Here we set the tradeoff to 0.5 for the microbes and fix the host maintenance at 1.

In [ ]:
sol = com.cooperative_tradeoff(min_growth={"colon": 1}, fraction=0.5, fluxes=True)

In [ ]:
# Check the abundance and growth rate of the model
sol.members

let's check butyrate fluxes in the two solutions.

In [ ]:
from micom.workflows.results import GrowthResults

res = GrowthResults.from_solution(sol, com)

In [ ]:
res.exchanges[res.exchanges.metabolite == "but[e]"]

We can chekc also what is passed to the host.

In [ ]:
res.exchanges[res.exchanges.taxon == "colon"].sort_values("flux")

Butyrate balance:

In [ ]:
res.exchanges[res.exchanges.metabolite.str.startswith("but")]

We can also calculate the ratio of butyrate taken up by the host.

In [ ]:
def butyrate_host_ratio(sol):
    """The ratio of butyrate taken up by the host."""
    but = sol.exchanges[sol.exchanges.metabolite.str.startswith("but[")]
    total = but[(but.metabolite == "but[e]") & (but.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = but[but.taxon == "colon"].flux.abs()
    return host.sum() / total

def butyrate_host_abs(sol):
    """The ratio of butyrate taken up by the host."""
    but = sol.exchanges[sol.exchanges.metabolite.str.startswith("but[")]
    total = but[(but.metabolite == "but[e]") & (but.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = but[but.taxon == "colon"].flux.abs()
    return host

for r in [res]:
    print(f"Ratio is: {100 * butyrate_host_ratio(r):.3f}%.")
    print("Abs is:", butyrate_host_abs(r))

In [ ]:
def acetate_host_ratio(sol):
    """The ratio of acetate taken up by the host."""
    ac = sol.exchanges[sol.exchanges.metabolite.str.startswith("ac[")]
    total = ac[(ac.metabolite == "ac[e]") & (ac.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = ac[ac.taxon == "colon"].flux.abs()
    return host.sum() / total

def acetate_host_abs(sol):
    """The ratio of acetate taken up by the host."""
    ac = sol.exchanges[sol.exchanges.metabolite.str.startswith("ac[")]
    total = ac[(ac.metabolite == "ac[e]") & (ac.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = ac[ac.taxon == "colon"].flux.abs()
    return host

for r in [res]:
    print(f"Ratio is: {100 * acetate_host_ratio(r):.3f}%.")
    print("Abs is:", acetate_host_abs(r))

In [ ]:
def propionate_host_ratio(sol):
    """The ratio of propionate taken up by the host."""
    ppa = sol.exchanges[sol.exchanges.metabolite.str.startswith("ppa[")]
    total = ppa[(ppa.metabolite == "ppa[e]") & (ppa.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = ppa[ppa.taxon == "colon"].flux.abs()
    return host.sum() / total

def propionate_host_abs(sol):
    """The ratio of propionate taken up by the host."""
    ppa = sol.exchanges[sol.exchanges.metabolite.str.startswith("ppa[")]
    total = ppa[(ppa.metabolite == "ppa[e]") & (ppa.direction == "export")]
    total = (total.abundance * total.flux).sum()
    host = ppa[ppa.taxon == "colon"].flux.abs()
    return host.sum() / total

for r in [res]:
    print(f"Ratio is: {100 * propionate_host_ratio(r):.3f}%.")
    print("Abs is:", propionate_host_abs(r))